# Problem: Find the Last Person Who Can Board the Bus

#### Table: Queue

| Column Name  | Type    |
|--------------|---------|
| person_id    | int     |
| person_name  | varchar |
| weight       | int     |
| turn         | int     |

- `person_id` column contains unique values.
- This table has the information about all people waiting for a bus.
- The `person_id` and `turn` columns will contain all numbers from `1` to `n`, where `n` is the number of rows in the table.
- `turn` determines the order of which the people will board the bus, where `turn=1` denotes the first person to board and `turn=n` denotes the last person to board.
- `weight` is the weight of the person in kilograms.

### Task

There is a queue of people waiting to board a bus. However, the bus has a **weight limit of 1000 kilograms**, so there may be some people who cannot board.

Write a solution to find the `person_name` of the last person that can fit on the bus without exceeding the weight limit. 

The test cases are generated such that the **first person does not exceed the weight limit**. Only one person can board the bus at any given turn.

### Example

### Input

#### Queue Table:

| person_id | person_name | weight | turn |
|-----------|-------------|--------|------|
| 5         | Alice       | 250    | 1    |
| 4         | Bob         | 175    | 5    |
| 3         | Alex        | 350    | 2    |
| 6         | John Cena   | 400    | 3    |
| 1         | Winston     | 500    | 6    |
| 2         | Marie       | 200    | 4    |

### Output

| person_name |
|-------------|
| John Cena   |

### Explanation

The following table is ordered by the `turn` column for simplicity:

| Turn | ID | Name       | Weight | Total Weight | Notes                  |
|------|----|------------|--------|--------------|------------------------|
| 1    | 5  | Alice      | 250    | 250          |                        |
| 2    | 3  | Alex       | 350    | 600          |                        |
| 3    | 6  | John Cena  | 400    | 1000         | (last person to board) |
| 4    | 2  | Marie      | 200    | 1200         | (cannot board)         |
| 5    | 4  | Bob        | 175    | -            | (cannot board)         |
| 6    | 1  | Winston    | 500    | -            | (cannot board)         |

In [10]:
import sqlite3
import pandas as pd

In [3]:
# Connect to a database (or create it if it doesn't exist)
conn = sqlite3.connect('example.db')

In [4]:
# Create a cursor object
cursor = conn.cursor()

In [5]:
# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Queue (
    person_id int,
    person_name varchar(30),
    weight int,
    turn int
)
''')

In [ ]:
# Delete all existing rows
cursor.execute('DELETE FROM Queue')

# Create data
data_to_insert = [
    ('5', 'Alice', '250', '1'),
    ('4', 'Bob', '175', '5'),
    ('3', 'Alex', '350', '2'),
    ('6', 'John Cena', '400', '3'),
    ('1', 'Winston', '500', '6'),
    ('2', 'Marie', '200', '4'),
]

# Insert data
cursor.executemany('''
INSERT INTO Queue (person_id, person_name, weight, turn)
VALUES (?, ?, ?, ?)
''', data_to_insert)

# Commit changes
conn.commit()

In [ ]:
# Query
query = '''
WITH TotalWeight AS (
    SELECT person_name,
        SUM(Weight) OVER (ORDER BY turn) AS total_weight
    FROM Queue
)
SELECT person_name
FROM TotalWeight
WHERE total_weight <= 1000
ORDER BY total_weight DESC
LIMIT 1
'''

# Query data
df = pd.read_sql_query(query, conn)

# Display the result as a query
df

,person_name
0,John Cena


In [25]:
# Close the connection
conn.close()